In [5]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 51 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619419 sha256=52f4341092cc11c89b14b6c577b0cbce62cc75249d769892fc063e6e91d71b48
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import Library

I will use surprise library for collaborative filtering in this task.

In [6]:
from surprise import SVD,Reader,Dataset
from surprise.model_selection import train_test_split

In [7]:
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# Import Dataset

In [11]:
data=pd.read_csv('/content/drive/MyDrive/Study/Groceries_data.csv')
data=data[['Member_number', 'itemDescription']]
data

,Member_number,itemDescription
0,1808,tropical fruit
1,2552,whole milk
2,2300,pip fruit
3,1187,other vegetables
4,3037,whole milk
...,...,...
38760,4471,sliced cheese
38761,2022,candy
38762,1097,cake bar
38763,1510,fruit/vegetable juice


# Preprocessing

In [12]:
#Rename columns and group by times that a customer buy an item
Data=data.groupby(data.columns.tolist(),as_index=False).size()
Data.columns=['member','item','times']
Data

,member,item,times
0,1000,canned beer,1
1,1000,hygiene articles,1
2,1000,misc. beverages,1
3,1000,pastry,1
4,1000,pickled vegetables,1
...,...,...,...
34761,5000,onions,1
34762,5000,other vegetables,1
34763,5000,root vegetables,1
34764,5000,semi-finished bread,1


In [13]:
#Remove member with less than or equal to 80 purchases
n=Data.member.value_counts(ascending=True).head(500)

In [14]:
mem=list(Data["member"].unique())

In [15]:
remove_memdf=n.reset_index()
remove_mem=list(remove_memdf['index'])

In [16]:
for i in remove_mem:
    mem.remove(i)

In [17]:
newData=Data.loc[Data['member'].isin(mem)]

In [18]:
newData.item.value_counts()

whole milk               1710
other vegetables         1386
rolls/buns               1299
soda                     1158
yogurt                   1053
                         ... 
frozen chicken              4
bags                        4
baby cosmetics              2
kitchen utensil             1
preservation products       1
Name: item, Length: 167, dtype: int64

In [19]:
#Remove item with less or equal to 100 purchases
n2=Data.item.value_counts(ascending=True).head(90)

In [20]:
items=list(Data["item"].unique())

In [21]:
remove_itemdf=n2.reset_index()
remove_item=list(remove_itemdf['index'])

In [22]:
for i in remove_item:
    items.remove(i)

In [23]:
newData=Data.loc[Data['item'].isin(items)]

We obtain a dataset with customers who have more than 80 purchases and items which is bought more than 100 times.

In [24]:
#Visualize the user-item matrix
from scipy.sparse import csr_matrix
df = newData.pivot(
    index='member',
    columns='item',
    values='times'
).fillna(0)
df

item,UHT-milk,baking powder,beef,berries,beverages,bottled beer,bottled water,brown bread,butter,butter milk,candy,canned beer,canned fish,cat food,chewing gum,chicken,chocolate,citrus fruit,coffee,cream cheese,curd,dessert,detergent,dishes,domestic eggs,flour,frankfurter,frozen fish,frozen meals,frozen vegetables,fruit/vegetable juice,grapes,ham,hamburger meat,hard cheese,herbs,hygiene articles,ice cream,liquor,long life bakery product,margarine,meat,misc. beverages,napkins,newspapers,oil,onions,other vegetables,packaged fruit/vegetables,pasta,pastry,pickled vegetables,pip fruit,pork,pot plants,processed cheese,red/blush wine,rolls/buns,root vegetables,salty snack,sausage,seasonal products,semi-finished bread,shopping bags,sliced cheese,soda,soft cheese,specialty bar,specialty chocolate,sugar,tropical fruit,waffles,whipped/sour cream,white bread,white wine,whole milk,yogurt
member,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
1001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# SVD Model

In [25]:
Reader=Reader(rating_scale=(0,6))
data = Dataset.load_from_df(newData[['member', 'item', 'times']], Reader)

I will use GridSearchCV to find the best parameters for the model.

In [26]:
param_grid = {
    'n_factors':[5,10,20],
    'n_epochs': [5,10,20], 
    'lr_all': [0.005,0.01]}

In [27]:
model = GridSearchCV(algo_class = SVD, param_grid = param_grid, n_jobs = -1, joblib_verbose = 5)

In [28]:
model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   54.0s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.2min finished


In [29]:
model.best_params

{'mae': {'lr_all': 0.01, 'n_epochs': 5, 'n_factors': 5},
 'rmse': {'lr_all': 0.005, 'n_epochs': 5, 'n_factors': 5}}

Fit model using the best parameters.

In [30]:
trainset, testset = train_test_split(data, test_size=0.2)

In [31]:
SVDmodel = SVD(n_factors= 5, n_epochs= 5, lr_all= 0.005)
SVDmodel.fit(trainset)
predictions = SVDmodel.test(testset)


Evaluate by RMSE.

In [32]:
accuracy.rmse(predictions)

RMSE: 0.3733


0.37332469969030085

# Recommendation Function

In [33]:
#print out 10 most recommended items for an input user
def make_rec(user):
    scores={}
    for item in items:
        scores[item]=SVDmodel.predict(uid = user, iid = item).est
        sorted_scores=dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))
    for _ in range(10):
        print("{}. {}".format(_+1, list(sorted_scores.keys())[_]))
    return None    

In [34]:
#recommended list for user 1000
make_rec(1000)

1. whole milk
2. other vegetables
3. rolls/buns
4. soda
5. yogurt
6. root vegetables
7. pastry
8. shopping bags
9. sausage
10. hamburger meat


In [35]:
#recommended list for user 1000
make_rec(5000)

1. whole milk
2. other vegetables
3. rolls/buns
4. soda
5. yogurt
6. root vegetables
7. sausage
8. tropical fruit
9. citrus fruit
10. pastry
